In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

from predict import Predict

In [2]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
test = list(df['PLAYER'].unique()[:100])
data = df[df['PLAYER'].isin(test)]
data

,GAME_ORDER,YEAR,DATE,GAME_NUMBER,WEEK,PLAYER,AGE,TEAM,OPPONENT,RESULT,...,PASSING_CMP_PCT,PASSING_YDS,PASSING_TD,PASSING_INT,PASSING_RATE,PASSING_SK,PASSING_YDS1,PASSING_Y_A,PASSING_AY_A,PPR
0,0.0,2018.0,2018-09-09,1.0,1.0,Christian McCaffrey,22.094,CAR,DAL,W 16-8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.5
1,1.0,2018.0,2018-09-16,2.0,2.0,Christian McCaffrey,22.101,CAR,ATL,L 24-31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.9
2,2.0,2018.0,2018-09-23,3.0,3.0,Christian McCaffrey,22.108,CAR,CIN,W 31-21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.4
3,3.0,2018.0,2018-10-07,4.0,5.0,Christian McCaffrey,22.122,CAR,NYG,W 33-31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.3
4,4.0,2018.0,2018-10-14,5.0,6.0,Christian McCaffrey,22.129,CAR,WAS,L 17-23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576,84.0,2023.0,2023-11-05,8.0,9.0,Amari Cooper,29.141,CLE,ARI,W 27-0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.9
4577,85.0,2023.0,2023-11-12,9.0,10.0,Amari Cooper,29.148,CLE,BAL,W 33-31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.8
4578,86.0,2023.0,2023-11-19,10.0,11.0,Amari Cooper,29.155,CLE,PIT,W 13-10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.4
4579,87.0,2023.0,2023-11-26,11.0,12.0,Amari Cooper,29.162,CLE,DEN,L 12-29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6


In [3]:
w = Predict(df=data, players = test, steps=17, lags=17)
df_res = w.predict_all()

c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [4]:
df_res

,PLAYER,ACTUAL SEASON TOTAL,PREDICTED,SEASON MAPE,GAME MAPE,GAMES PREDICTED,LAGS,REGRESSOR
0,Christian McCaffrey,416.20,403.0438,0.031610,0.275009,17,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
1,Tyreek Hill,388.90,311.2650,0.199627,0.678218,17,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
2,CeeDee Lamb,363.80,228.2040,0.372721,0.399622,17,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
3,Josh Allen,436.86,429.7798,0.016207,0.217679,17,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
4,Raheem Mostert,276.50,73.4160,0.734481,0.625547,17,17,"(DecisionTreeRegressor(max_features=1.0, rando..."
...,...,...,...,...,...,...,...,...
70,Zach Wilson,203.60,253.2952,0.244083,0.860423,17,3,"(DecisionTreeRegressor(max_features=1.0, rando..."
71,Austin Ekeler,299.70,346.5560,0.156343,0.698498,17,3,"(DecisionTreeRegressor(max_features=1.0, rando..."
72,Christian Kirk,210.56,218.8410,0.039328,0.847906,17,3,"(DecisionTreeRegressor(max_features=1.0, rando..."
73,Jaylen Waddle,221.50,238.7520,0.077887,0.840679,17,3,"(DecisionTreeRegressor(max_features=1.0, rando..."


In [5]:
np.mean(df_res['SEASON MAPE']), np.mean(df_res['GAME MAPE'])

(0.2326667301778834, 225701339254800.1)